In [25]:
#!/usr/bin/env python

from __future__ import division
%matplotlib inline

import tensornetwork as tn
import numpy as np
import scipy as sp

In [26]:
def genrandarr():
    num=[]
    num.append(np.random.rand(p,D))
    for i in range(1,N-1):
        num.append(np.random.rand(p,D,D))
    num.append(np.random.rand(p,D))
    return num

In [27]:
def create2MPSs(num):    
    loc1=[]
    loc2=[]
    for i in range(N):
        loc1.append(tn.Node(num[i]))
        loc2.append(tn.Node(np.conj(num[i])))
    
    loc1[0][1] ^ loc1[1][1]
    for i in range (1,N-1):
        loc1[i][2] ^ loc1[i+1][1]
    loc2[0][1] ^ loc2[1][1]
    for i in range (1,N-1):
        loc2[i][2] ^ loc2[i+1][1]
    
    return [loc1,loc2]

In [28]:
def collapsepartPsiPsi(mps1,mps2,u,v):
    y=[]
    for i in range(u,v+1):
        y.append(mps1[i] @ mps2[i])
    z=shrink(y,v-u+1)
    return z

In [29]:
def collapsepartPsiSigmaSigmaPsi(mps1,mps2,s1,s2,k,u,v):
    y=[]
    if k!=N-1:
        g1=k
        g2=k+1
    else:
        g1=N-1
        g2=0
    for i in range(u,v+1):
        if i!=g1 and i!=g2:
            y.append(mps1[i] @ mps2[i])
        elif i==g1:
            y.append(mps1[i] @ s1 @ mps2[i])
        else:
            y.append(mps1[i] @ s2 @ mps2[i])
    z=shrink(y,v-u+1)
    return z

In [30]:
def collapsepartPsiSigmaPsi(mps1,mps2,s,k,u,v):
    y=[]
    for i in range(u,v+1):
        if i!=k:
            y.append(mps1[i] @ mps2[i])
        else:
            y.append(mps1[i] @ s @ mps2[i])
    z=shrink(y,v-u+1)
    return z

In [31]:
def shrink(mps,length):
    if length!=0: #можно убрать, т.к. вызываем shrink только в таком случае
        res = mps[0]
        for i in range(1,length):
            res = res @ mps[i]
        return res

In [32]:
def createSigmaNode(ind):
    if ind==0:
        sigmanode = tn.Node(np.array([[1,0],[0,1]],dtype=np.complex_))
    elif ind==1:
        sigmanode = tn.Node(np.array([[0,1],[1,0]],dtype=np.complex_))
    elif ind==2:
        sigmanode = tn.Node(np.array([[0,-1j],[1j,0]],dtype=np.complex_))
    elif ind==3:
        sigmanode = tn.Node(np.array([[1,0],[0,-1]],dtype=np.complex_))
    return sigmanode

In [33]:
def createPsiSigmaSigmaPsi(num,ind,k):
    s1 = createSigmaNode(ind)
    s2 = createSigmaNode(ind)
    mpss=create2MPSs(num)
    mps1=mpss[0]
    mps2=mpss[1]
    if k!=N-1:
        g1=k
        g2=k+1
    else:
        g1=N-1
        g2=0
    for i in range(N):
        if i!=g1 and i!=g2:
            mps1[i][0] ^ mps2[i][0]
        elif i==g1:
            mps1[i][0] ^ s1[0]
            s1[1] ^ mps2[i][0]
        else:
            mps1[i][0] ^ s2[0]
            s2[1] ^ mps2[i][0]
    return [mps1,mps2,s1,s2]

In [34]:
def createPsiSigmaPsi(num,ind,k):
    s = createSigmaNode(ind)
    mpss=create2MPSs(num)
    mps1=mpss[0]
    mps2=mpss[1]
    for i in range(N):
        if i!=k:
            mps1[i][0] ^ mps2[i][0]
        else:
            mps1[i][0] ^ s[0]
            s[1] ^ mps2[i][0]
    return [mps1,mps2,s]

In [35]:
def createPsiPsi(num):
    mpss=create2MPSs(num)
    mps1=mpss[0]
    mps2=mpss[1]
    for i in range(N):
        mps1[i][0] ^ mps2[i][0]
    return [mps1,mps2]

In [36]:
def separateAfromPsiPsi(mps1,mps2,l):
    AA=mps1[l].tensor
    BB=mps2[l].tensor
    iden=np.identity(p,dtype=np.complex_)
    if l==0:
        r = collapsepartPsiPsi(mps1,mps2,1,N-1)
        bb=r.tensor
        return vecmatrveccorner(AA,BB,iden,bb)
    elif l==N-1:
        r = collapsepartPsiPsi(mps1,mps2,0,N-2)
        aa=r.tensor
        return vecmatrveccorner(AA,BB,iden,aa)
    else:
        r1 = collapsepartPsiPsi(mps1,mps2,0,l-1)
        r2 = collapsepartPsiPsi(mps1,mps2,l+1,N-1)
        aa=r1.tensor
        cc=r2.tensor
        return vecmatrvecmiddle(AA,BB,iden,aa,cc)

In [37]:
def separateAfromPsiSigmaSigmaPsi(mps1,mps2,s1,s2,k,l):
    if k!=N-1:
        g1=k
        g2=k+1
    else:
        g1=N-1
        g2=0
    AA=mps1[l].tensor
    BB=mps2[l].tensor
    if g1==l:            
        op=s1.tensor
    elif g2==l:
        op=s2.tensor
    else:
        op=np.identity(p,dtype=np.complex_)
    
    if l==0:
        r = collapsepartPsiSigmaSigmaPsi(mps1,mps2,s1,s2,k,1,N-1)
        bb=r.tensor
        return vecmatrveccorner(AA,BB,op,bb)
    elif l==N-1:
        r = collapsepartPsiSigmaSigmaPsi(mps1,mps2,s1,s2,k,0,N-2)
        aa=r.tensor
        return vecmatrveccorner(AA,BB,op,aa)
    else:
        r1 = collapsepartPsiSigmaSigmaPsi(mps1,mps2,s1,s2,k,0,l-1)
        r2 = collapsepartPsiSigmaSigmaPsi(mps1,mps2,s1,s2,k,l+1,N-1)
        aa=r1.tensor
        cc=r2.tensor
        return vecmatrvecmiddle(AA,BB,op,aa,cc)

In [38]:
def separateAfromPsiSigmaPsi(mps1,mps2,s,k,l):
    AA=mps1[l].tensor
    BB=mps2[l].tensor
    if k==l:            
        op=s.tensor
    else:
        op=np.identity(p,dtype=np.complex_)
    
    if l==0:
        r = collapsepartPsiSigmaPsi(mps1,mps2,s,k,1,N-1)
        bb=r.tensor
        return vecmatrveccorner(AA,BB,op,bb)
    elif l==N-1:
        r = collapsepartPsiSigmaPsi(mps1,mps2,s,k,0,N-2)
        aa=r.tensor
        return vecmatrveccorner(AA,BB,op,aa)
    else:
        r1 = collapsepartPsiSigmaPsi(mps1,mps2,s,k,0,l-1)
        r2 = collapsepartPsiSigmaPsi(mps1,mps2,s,k,l+1,N-1)
        aa=r1.tensor
        cc=r2.tensor
        return vecmatrvecmiddle(AA,BB,op,aa,cc)

In [39]:
def vecmatrvecmiddle(A,B,a0,a1,a2):
    q=D*p*D
    F = np.zeros(q,dtype=np.complex_)
    G = np.zeros((q,q),dtype=np.complex_)
    H = np.zeros(q,dtype=np.complex_)
    I=0
    J=0
    for i in range(p):
        for j in range(D):
            for k in range(D):
                F[I] = A[i][j][k]
                H[I] = B[i][j][k]
                for l in range(p):
                    for m in range(D):
                        for n in range(D):
                            G[I][J] = a0[i][l]*a1[j][m]*a2[k][n]
                            J=J+1
                I=I+1
                J=0
    return [F,G,H]

In [40]:
def vecmatrveccorner(A,B,a0,a1):
    q=p*D
    F = np.zeros(q,dtype=np.complex_)
    G = np.zeros((q,q),dtype=np.complex_)
    H = np.zeros(q,dtype=np.complex_)
    I=0
    J=0
    for i in range(p):
        for j in range(D):
            F[I] = A[i][j]
            H[I] = B[i][j]
            for l in range(p):
                for m in range(D):
                    G[I][J] = a0[i][l]*a1[j][m]
                    J=J+1
            I=I+1
            J=0
    return [F,G,H]

In [41]:
def getHeff(m,l):
    if l==0 or l==N-1:
        q=p*D
    else:
        q=D*p*D
    G = np.zeros((q,q),dtype=np.complex_)
    for k in range(N):
        pzzp=createPsiSigmaSigmaPsi(m,3,k)
        pzp=createPsiSigmaPsi(m,3,k)
        pxp=createPsiSigmaPsi(m,1,k)
        tzz=separateAfromPsiSigmaSigmaPsi(pzzp[0],pzzp[1],pzzp[2],pzzp[3],k,l)
        tz=separateAfromPsiSigmaPsi(pzp[0],pzp[1],pzp[2],k,l)
        tx=separateAfromPsiSigmaPsi(pxp[0],pxp[1],pxp[2],k,l)
        G = G - tzz[1] + h*tz[1] + g*tx[1]
    return G

In [42]:
def getNop(m,l):
    q=D*p*D
    pp=createPsiPsi(m)
    t=separateAfromPsiPsi(pp[0],pp[1],l)
    return [t[0],t[1]]

In [43]:
def Avectomatr(vec,l):
    I=0
    if l==0 or l==N-1:
        matr = np.zeros((p,D),dtype=np.complex_)
        for i in range(p):
            for j in range(D):
                matr[i][j]=vec[I]
                I=I+1
    else:
        matr = np.zeros((p,D,D),dtype=np.complex_)
        for i in range(p):
            for j in range(D):
                for k in range(D):
                    matr[i][j][k]=vec[I]
                    I=I+1
    return matr

In [44]:
def calcEnergy(num):
    E = 0
    for k in range(N):        
        pzzp=createPsiSigmaSigmaPsi(num,3,k)
        pzp=createPsiSigmaPsi(num,3,k)
        pxp=createPsiSigmaPsi(num,1,k)
        tzz=collapsepartPsiSigmaSigmaPsi(pzzp[0],pzzp[1],pzzp[2],pzzp[3],k,0,N-1)
        tz=collapsepartPsiSigmaPsi(pzp[0],pzp[1],pzp[2],k,0,N-1)
        tx=collapsepartPsiSigmaPsi(pxp[0],pxp[1],pxp[2],k,0,N-1)
        E = E - tzz.tensor + h*tz.tensor + g*tx.tensor
    return E

In [45]:
N = 12 #длина цепочки
D = 1 #bond index
p = 2 #выходящий индекс
itnum = 10000 #число итераций для нахождения ground state
avnum = 3

In [46]:
g = 1.05

In [ ]:
for a in range(avnum):
    h = 0
    for ii in range(11):
        print('h =',h)
        m=genrandarr()
        for i in range(itnum):
            aloc=np.random.randint(N)
            e=getNop(m,aloc)
            A=e[0]
            Nop=e[1]
            Heff=getHeff(m,aloc)
            u=sp.linalg.eig(Heff,Nop)[1][0]
            v=Avectomatr(u,aloc)
            m[aloc]=v
        EnCur = calcEnergy(m)
        print('En =',EnCur)
        h=h+0.1

h = 0
En = (-2.077499999999996+0j)
h = 0.1
En = (-0.21987959405813007+0j)
h = 0.2
En = (1.8000000000000014+0j)
h = 0.30000000000000004
En = (4.122496783422224+0j)
h = 0.4
En = (-3.5263686417938773+0j)
h = 0.5
En = (0.9351206631223583+0j)
h = 0.6
En = (-0.6170797476084959+0j)
h = 0.7
En = (-2.1730307067897625+0j)
h = 0.7999999999999999
En = (2.815729163563857+0j)
h = 0.8999999999999999
En = (1.3254856744954382+0j)
h = 0.9999999999999999
En = (4.384650126025523+0j)
h = 0
En = (-2.077499999999996+0j)
h = 0.1
En = (-0.21987959405812985+0j)
h = 0.2
En = (1.8000000000000003+0j)
h = 0.30000000000000004
En = (4.1224967834222275+0j)
h = 0.4
En = (-3.570394289351608+0j)
h = 0.5
En = (-0.9931273204198892+0j)
h = 0.6
En = (-0.6110346201112321+0j)
h = 0.7
En = (0.1620939195045168+0j)
h = 0.7999999999999999
En = (2.8393491327086395+0j)
h = 0.8999999999999999
En = (3.55747391577484+0j)
h = 0.9999999999999999
